# This notebooks calls the edgefitting_2D.py routine and makes some pictures afterwards

In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import AdvancedBraggEdgeFitting
from PIL import Image

import tifffile
from tifffile import TiffFile

from skimage import io

import os, fnmatch
from os import listdir

from TOF_routines import tof2l
from TOF_routines import find_nearest

from astropy.io import fits
import edgefitting_2D

import ipywe.fileselector

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
#TODO : remove the unused libs

%matplotlib notebook

### Get the path data 

In [ ]:
fsel_data= ipywe.fileselector.FileSelectorPanel(instruction='select directory for data' , start_dir='.', type='directory')
fsel_data.show()
#ex. '/media/carminati_c/Data2/LSP_Manuel/sample_binned_spotCleaned/'

### Get the path OB 

In [ ]:
fsel_ob= ipywe.fileselector.FileSelectorPanel(instruction='select directory for open beam' , start_dir='.', type='directory')
fsel_ob.show()
#ex. '/media/carminati_c/Data2/LSP_Manuel/OB_binned_spotCleaned/'

### Get the spectrum txt file

In [ ]:
fsel_spectrum = ipywe.fileselector.FileSelectorPanel(instruction='select Spectra.txt' , start_dir='.', type='file')
fsel_spectrum.show()
#ex. '/media/ws_niag/10_people/Morgano/RADEN_data_analysis/TEST6_000_Spectra.txt

### Get the mask file 

In [ ]:
fsel_mask = ipywe.fileselector.FileSelectorPanel(instruction='select mask file' , start_dir='.', type='file')
fsel_mask.show()
#ex. /home/carminati_c/git/ToFImaging/scripts/mymask.tif

### Define the parameters needed for the edgefitting_2D function 

In [ ]:
pathdata = fsel_data.selected
pathob = fsel_ob.selected
filemask = fsel_mask.selected
files_sample = (sorted(fnmatch.filter(listdir(pathdata),'*.tif')))
files_ob = (sorted(fnmatch.filter(listdir(pathob),'*.tif')))
spectrum = fsel_spectrum.selected

In [ ]:
# These numbers come from TOF_Calibration
t0=-0.0002618673892937752 
L = 18.5
cal_parameters = np.array([t0, L])
#other initialization parameters
binning = 18
lambda_range = np.array([3.7, 4.4])
est_parameters = np.array([0.1, 0.1, 4.05]) # sigma, alpha, edge_position

In [ ]:
res = edgefitting_2D.image_edge_fitting(pathdata, pathob, filemask, spectrum, cal_parameters, binning, lambda_range, est_parameters)

In [ ]:
edge_pos = np.

### Example how to compute initial parameters: 

In [ ]:
files_ob = (sorted(fnmatch.filter(listdir(pathob),'*.tif')))
# print(files_ob)
ob = io.imread(pathob+'/'+files_ob[0])
np.array([np.shape(ob)[0], np.shape(ob)[1], len(files_ob)])

In [ ]:
#load spectrum, bin, and convert to lambda
spec = np.loadtxt(spectrum, usecols=0)
spectrum_binned = spec[0::18]
print(len(spectrum_binned))
mylambda_bin = tof2l(spectrum_binned,t0,L)

In [ ]:
#load ob and sample images
print(pathob+'/'+files_ob[0])

ob = io.imread(pathob+'/'+files_ob[0])
np.shape(ob)

ob_image = np.zeros([300,400,155])
sample_image = np.zeros([300,400,155])
np.shape(ob_image)

for i in range(0,155):
    ob_image[:,:,i] = io.imread(pathob+'/'+files_ob[i])
    sample_image[:,:,i] = io.imread(pathdata+'/'+files_sample[i])

In [ ]:
# compute transmission
trans_image = np.zeros([300,400,155])
for i in range(0,155):
    dose = np.median(ob_image[10:20,10:20,i])/np.median(sample_image[10:20,10:20,i])
    trans_image[:,:,i] = sample_image[:,:,i]/ob_image[:,:,i]*dose

In [ ]:
plt.figure()
plt.imshow(trans_image[:,:,70],cmap='gray')
plt.axis('off')
# plt.savefig('transmission_lsp.pdf')

In [ ]:
#compute spectrum over a large region to compute test initial parameters
sp=np.zeros(156)
for i in range(0,155):
    sp[i] = np.median(trans_image[37,254,i])

In [ ]:
plt.figure()
plt.plot(mylambda_bin, sp)
plt.title('total spectrum')

In [ ]:
%%time
myrange =[]
myrange.append(find_nearest(mylambda_bin, 3.7))
myrange.append(find_nearest(mylambda_bin, 4.4))
print(myrange)

plt.figure()
plt.plot(mylambda_bin[myrange[0]:myrange[1]], sp[myrange[0]:myrange[1]])

est_sigma = 0.1
est_alpha = 0.1
est_pos = find_nearest(mylambda_bin, 4.05)
print(est_pos)
ris = AdvancedBraggEdgeFitting.AdvancedBraggEdgeFitting(sp, myrange, mylambda_bin, est_pos, est_sigma, est_alpha, True, False, False, True)
# t_edg = ris['t0']
# print(t_edg)

### This is an example how to create a mask

In [ ]:
# This in an example how to create the mask
ob = io.imread(pathob+'/'+files_ob[0])
np.shape(ob)

ob_image = np.zeros([300,400,155])
sample_image = np.zeros([300,400,155])
np.shape(ob_image)

for i in range(0,155):
    ob_image[:,:,i] = io.imread(pathob+'/'+files_ob[i])
    sample_image[:,:,i] = io.imread(pathdata+'/'+files_sample[i])


trans_image = np.zeros([300,400,155])
for i in range(0,155):
    dose = np.median(ob_image[10:20,10:20,i])/np.median(sample_image[10:20,10:20,i])
    trans_image[:,:,i] = sample_image[:,:,i]/ob_image[:,:,i]*dose

mymask = trans_image[:,:,70]<0.5

mymask[:28,:] = False
mymask[:,380:] = False

plt.figure()
plt.imshow(mymask)
plt.axis('off')
plt.axis('tight')
# io.imsave('mymask.tif', mymask+0)

### Example on data plotting after 2D edge fitting 

In [ ]:
test = np.load('edge_position.npy')
plt.figure()
plt.imshow(test,vmin=4.10, vmax=4.15)
plt.colorbar()

In [ ]:
# Check the number of points where the fitting fails
sum(sum(n < 0 for n in EP))

In [ ]:
# Re-run the fitting in those points

#     t0:                 4.07855978 +/- 15750.9720 (386189.56%) (init = 4.076283)
#     alpha:              8.8506e-04 +/- 26892.9752 (3038556174.03%) (init = 0.007669497)
#     sigma:              2.0660e-04 +/- 62396.2248 (30201102063.85%) (init = 0.0001408172)
#     a1:                 1.94456283 +/- 1.28224664 (65.94%) (init = 0.9776328)
#     a2:                -0.03410872 +/- 0.30144455 (883.78%) (init = 0.1905023)
#     a5:                -4.32338756 +/- 1.99813278 (46.22%) (init = -3.757007)
#     a6:                 1.29564172 +/- 0.49946988 (38.55%) (init = 1.176675)
        
est_alpha = 0.1
est_sigma = 0.1
a1 = 1.94456283
a2 = -0.03410872
a5 = -4.32338756
a6 = 1.29564172
est_pos = find_nearest(mylambda_bin[myrange[0]:myrange[1]], 4.07)

missing_edges = np.zeros(np.shape(mymask))
missing_heigths = np.zeros(np.shape(mymask))
print(est_pos)

print(myrange)
print(small_range)


# Re-run the edge fitting where the first set of parameters failed
for i in range(0, np.shape(mymask)[0]):
    
    for j in range(0, np.shape(mymask)[1]):
        
        if (EP[i,j]==-2):
            # extract the signal
            mysignal = np.zeros(myrange[1]-myrange[0])
            
    
            for ind in range(myrange[0],myrange[1]):
                mysignal[ind-myrange[0]] = np.median(trans_image[i,j,ind])
            
            try:
                edge_fit = AdvancedBraggEdgeFitting.AdvancedBraggEdgeFitting(mysignal, small_range, small_lambda, est_pos, est_sigma, est_alpha, False, False, False, True)
                missing_edges[i,j] = edge_fit['t0']
                missing_heigths[i,j] = edge_fit['height']
            except:
                print("Unexpected error at :", i, j)
                missing_edges[i,j]= -2.0
                missing_heigths[i,j] = -2.0

                

In [ ]:
plt.figure()
plt.imshow(missing_edges)

In [ ]:
# Run again for the further missing edges 
est_alpha = 0.09
est_sigma = 0.09
a1 = 1.94456283
a2 = -0.03410872
a5 = -4.32338756
a6 = 1.29564172
est_pos = find_nearest(mylambda_bin[myrange[0]:myrange[1]], 4.05)


for i in range(0, np.shape(mymask)[0]):
    
    for j in range(0, np.shape(mymask)[1]):
        
        if (missing_edges[i,j]==-2):
            # extract the signal
            mysignal = np.zeros(myrange[1]-myrange[0])
            
    
            for ind in range(myrange[0],myrange[1]):
                mysignal[ind-myrange[0]] = np.median(trans_image[i,j,ind])
            
            try:
                edge_fit = AdvancedBraggEdgeFitting.AdvancedBraggEdgeFitting(mysignal, small_range, small_lambda, est_pos, est_sigma, est_alpha, False, False, False, True)
                missing_edges[i,j] = edge_fit['t0']
                missing_heigths[i,j] = edge_fit['height']

            except:
                print("Unexpected error at :", i, j)
                missing_edges[i,j]= -2.0
                missing_heigths[i,j] = -2.0

In [ ]:
updated_EP = np.zeros(np.shape(mymask))

for i in range(0, np.shape(mymask)[0]):
    for j in range(0, np.shape(mymask)[1]):
        if (EP[i,j]!=-2):
            updated_EP[i,j]= EP[i,j]
        else:
            updated_EP[i,j]= missing_edges[i,j]
# updated_EP = EP[EP!=-2]
# updated_EP = missing_edges[EP==-2]

In [ ]:
plt.figure()
plt.imshow(np.rot90(updated_EP,3),vmin=4.05, vmax=4.09)
plt.axis('off')
plt.colorbar()
plt.savefig('LSP_model.pdf')

In [ ]:
#saving some results
np.save('edge_positions_all.npy', updated_EP)
fits.writeto('edge_pos_all.fits',updated_EP)